In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np
from torch import Tensor
import torch
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector



In [ ]:


def one_qubit_rotation(qc, weights, gate_types_per_layer, enable_rx=True, enable_ry=True, enable_rz=True):
    """
    Adds rotation gates around the X, Y, and Z axis to the quantum circuit for a specific qubit,
    with the rotation angles specified by the values in `weights`.
    """
    if len(weights) != gate_types_per_layer * len(qc.qubits):
        print(f"len weights {len(weights)}")
        print(f"gate types per layer {gate_types_per_layer}")
        print(f"len qubits {len(qc.qubits)}")
        print(f"len qubits * gate types per layer {len(qc.qubits) * gate_types_per_layer}")
        raise ValueError(
            "The number of weights must be equal to the number of qubits times the number of gate types per layer")

    if not enable_rx and not enable_ry and not enable_rz:
        raise ValueError("At least one gate must be enabled")

    enabled_gates = sum([enable_rx, enable_ry, enable_rz])  # Count how many gates are enabled
    if len(weights) != enabled_gates * len(qc.qubits):
        print(len(weights))
        print(enabled_gates)
        print(len(qc.qubits))
        print(len(qc.qubits) * enabled_gates)
        raise ValueError("The number of weights does not match the number of enabled gates times the number of qubits")

    for qubit in qc.qubits:
        index = qubit.index
        weight_index = 0  # Initialize weight index for each qubit

        if enable_rx:
            qc.rx(weights[index + weight_index * len(qc.qubits)], index)  # Rotate around X-axis
            weight_index += 1  # Move to the next set of weights

        if enable_ry:
            qc.ry(weights[index + weight_index * len(qc.qubits)], index)  # Rotate around Y-axis
            weight_index += 1  # Move to the next set of weights

        if enable_rz:
            qc.rz(weights[index + weight_index * len(qc.qubits)], index)  # Rotate around Z-axis
            # No need to increase weight_index here since it's the last operation


def entangling_layer(qc: QuantumCircuit):
    """
    Adds a layer of CZ entangling gates (controlled-Z) on `qubits` (arranged in a circular topology) to the quantum circuit.
    """
    # Assume `qc` is your QuantumCircuit object that's defined outside this function
    for i in range(qc.num_qubits - 1):
        qc.cz(i, i + 1)  # Apply CZ between consecutive qubits
    if qc.num_qubits > 2:  # If more than 2 qubits, connect the first and last qubits to form a circle
        qc.cz(0, qc.num_qubits - 1)




In [ ]:

def generate_circuit(qc, n_layers, enable_rx=True, enable_ry=True, enable_rz=True):
    """Prepares a data re-uploading circuit on `qubits` with `n_layers` layers."""
    # Number of qubits
    n_qubits = qc.num_qubits
    # if gate is enabled, add a parameter for each qubit and for each layer

    gate_types_per_layer = 0
    if enable_rx:
        gate_types_per_layer += 1
    if enable_ry:
        gate_types_per_layer += 1
    if enable_rz:
        gate_types_per_layer += 1
    if gate_types_per_layer == 0:
        raise ValueError("At least one gate must be enabled")
    if n_layers < 1:
        raise ValueError("At least one layer is required")
    if n_qubits < 1:
        raise ValueError("At least one qubit is required")

    params = ParameterVector("theta", gate_types_per_layer * (n_layers + 1) * n_qubits)
    inputs = ParameterVector("inputs", n_qubits)

    for i in range(n_layers):
        for j in range(n_qubits):
            qc.rx(inputs[j], j)
        qc.barrier()
        # Variational layer
        if i == 0:
            one_qubit_rotation(qc, params[0:gate_types_per_layer * n_qubits], gate_types_per_layer=gate_types_per_layer,
                               enable_rx=enable_rx, enable_ry=enable_ry, enable_rz=enable_rz)
        else:
            one_qubit_rotation(qc,
                               params[i * gate_types_per_layer * n_qubits:(i + 1) * gate_types_per_layer * n_qubits],
                               gate_types_per_layer=gate_types_per_layer, enable_rx=enable_rx, enable_ry=enable_ry,
                               enable_rz=enable_rz)
        qc.barrier()
        entangling_layer(qc)
        # Encoding layer
        qc.barrier()

    for i in range(n_qubits):
        qc.rx(inputs[i], i)
    one_qubit_rotation(qc, params[
                           n_layers * gate_types_per_layer * n_qubits:(n_layers + 1) * gate_types_per_layer * n_qubits],
                       gate_types_per_layer=gate_types_per_layer, enable_rx=enable_rx, enable_ry=enable_ry,
                       enable_rz=enable_rz)

    return qc, list(params), list(inputs)


In [ ]:

qc = QuantumCircuit(4)
qc, params, inputs = generate_circuit(qc, 8, enable_rx=True, enable_ry=True, enable_rz=True)
qc.draw("mpl", style="iqx")


In [ ]:
sampler_qnn = SamplerQNN(circuit=qc, input_params=inputs, weight_params=params)


In [ ]:
qnn = TorchConnector(sampler_qnn)


In [1]:
qnn

NameError: name 'qnn' is not defined

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class ReUploadingPQC(nn.Module):
    """
    Placeholder PyTorch implementation. Note that quantum circuit generation and ControlledPQC are not
    directly implemented, as they depend on the quantum framework used with PyTorch.
    """
    def __init__(self, qc, n_layers, observables, activation='linear', name="re-uploading_PQC"):
        super(ReUploadingPQC, self).__init__()
        self.n_layers = n_layers
        self.n_qubits = len(qc.num_qubits)
        
        # Placeholder for circuit generation
        circuit, theta_symbols, input_symbols = generate_circuit(qc, n_layers) # Needs implementation
        
        self.theta = nn.Parameter(
            torch.rand(1, len(theta_symbols)) * np.pi,  # Uniform initialization [0, pi]
            requires_grad=True
        )
        
        self.lmbd = nn.Parameter(
            torch.ones(self.n_qubits * self.n_layers),
            requires_grad=True
        )
        
        # Placeholder for defining symbol order and computation layer
        symbols = [str(symb) for symb in theta_symbols + input_symbols]
        self.indices = torch.tensor([symbols.index(a) for a in sorted(symbols)])
        
        if activation == 'linear':
            self.activation = nn.Identity()
        else:
            # Add more activation functions as needed
            self.activation = getattr(nn, activation.capitalize())()
        
        # Placeholder for ControlledPQC and empty circuit
        self.computation_layer = None  # Needs implementation
        
    def forward(self, inputs):
        batch_dim = inputs[0].shape[0]
        # Placeholder for repeating operations related to quantum circuits
        tiled_up_thetas = self.theta.repeat(batch_dim, 1)
        tiled_up_inputs = inputs[0].repeat(1, self.n_layers)
        scaled_inputs = tiled_up_inputs * self.lmbd
        
        squashed_inputs = self.activation(scaled_inputs)
        
        joined_vars = torch.cat([tiled_up_thetas, squashed_inputs], dim=1)
        joined_vars = joined_vars[:, self.indices]
        
        # Placeholder for computation with the quantum layer
        # return self.computation_layer([empty_circuits, joined_vars])
        return joined_vars  # Placeholder return



In [ ]:

class encoding_layer(torch.nn.Module):
    def __init__(self, num_qubits=4):
        super().__init__()

        # Define weights for the layer
        weights = torch.Tensor(num_qubits)
        self.weights = torch.nn.Parameter(weights)
        torch.nn.init.uniform_(self.weights, -1, 1)  # <--  Initialization strategy

    def forward(self, x):
        """Forward step, as explained above."""

        if not isinstance(x, Tensor):
            x = Tensor(x)

        x = self.weights * x
        x = torch.atan(x)

        return x


class exp_val_layer(torch.nn.Module):
    def __init__(self, action_space=2):
        super().__init__()

        # Define the weights for the layer
        weights = torch.Tensor(action_space)
        self.weights = torch.nn.Parameter(weights)
        torch.nn.init.uniform_(self.weights, 35, 40)  # <-- Initialization strategy (heuristic choice)

        # Masks that map the vector of probabilities to <Z_0*Z_1> and <Z_2*Z_3>
        self.mask_ZZ_12 = torch.tensor([1., -1., -1., 1., 1., -1., -1., 1., 1., -1., -1., 1., 1., -1., -1., 1.],
                                       requires_grad=False)
        self.mask_ZZ_34 = torch.tensor([-1., -1., -1., -1., 1., 1., 1., 1., -1., -1., -1., -1., 1., 1., 1., 1.],
                                       requires_grad=False)

    def forward(self, x):
        """Forward step, as described above."""

        expval_ZZ_12 = self.mask_ZZ_12 * x
        expval_ZZ_34 = self.mask_ZZ_34 * x

        # Single sample
        if len(x.shape) == 1:
            expval_ZZ_12 = torch.sum(expval_ZZ_12)
            expval_ZZ_34 = torch.sum(expval_ZZ_34)
            out = torch.cat((expval_ZZ_12.unsqueeze(0), expval_ZZ_34.unsqueeze(0)))

        # Batch of samples
        else:
            expval_ZZ_12 = torch.sum(expval_ZZ_12, dim=1, keepdim=True)
            expval_ZZ_34 = torch.sum(expval_ZZ_34, dim=1, keepdim=True)
            out = torch.cat((expval_ZZ_12, expval_ZZ_34), 1)

        return self.weights * ((out + 1.) / 2.)